# Multi-output neural networks

In [2]:
import numpy as np
import keras
from keras.layers import Dense, Input
from keras.models import Model
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical

### 1.  
General loss: 15.5   
Main problems:
- ATS and HD difference in buildings
- Regression tasks could be improved
Remarks:
- Categorical cross-entropy required one-hot encoded y values


In [2]:
X_train = pd.read_csv('X_train_C.csv', low_memory=False)
X_test = pd.read_csv('X_test_C.csv', low_memory=False)

Y_train = pd.read_csv('Y_train_C.csv', low_memory=False)
Y_test = pd.read_csv('Y_test_C.csv', low_memory=False)

In [38]:
input_shape = X_train.shape[1] # Number of features

# Define the input layer
input_layer = Input(shape=(input_shape,))

label_encoder = LabelEncoder()

Y_train['Nail spacing [cm]'] = label_encoder.fit_transform(Y_train['Nail spacing [cm]'])
Y_test['Nail spacing [cm]'] = label_encoder.transform(Y_test['Nail spacing [cm]'])

# Use to_categorical
Y_train_onehot = to_categorical(Y_train['Nail spacing [cm]'], num_classes=3)
Y_test_onehot = to_categorical(Y_test['Nail spacing [cm]'], num_classes=3)


# Define separate branches for each output
nail_spacing_output = Dense(3, activation='softmax', name='nail_spacing')(input_layer)
num_sheathing_panels_output = Dense(3, activation='softmax', name='num_sheathing_panels')(input_layer)
num_end_studs_output = Dense(6, activation='softmax', name='num_end_studs')(input_layer)
total_num_studs_output = Dense(1, activation='linear', name='total_num_studs')(input_layer)
holddown_model_output = Dense(1, activation='linear', name='holddown_model')(input_layer)
tx_output = Dense(1, activation='linear', name='tx')(input_layer)
ty_output = Dense(1, activation='linear', name='ty')(input_layer)

# Combine the outputs into a single model
model = Model(inputs=input_layer, outputs=[nail_spacing_output, num_sheathing_panels_output, 
                                                   num_end_studs_output, total_num_studs_output, 
                                                   holddown_model_output, tx_output, ty_output])

# Compile the model
model.compile(optimizer='adam', 
              loss={'nail_spacing': 'categorical_crossentropy',
                    'num_sheathing_panels': 'categorical_crossentropy',
                    'num_end_studs': 'categorical_crossentropy',
                    'total_num_studs': 'mean_squared_error',
                    'holddown_model': 'mean_squared_error',
                    'tx': 'mean_squared_error',
                    'ty': 'mean_squared_error'},
              metrics={'nail_spacing': 'accuracy',
                       'num_sheathing_panels': 'accuracy',
                       'num_end_studs': 'accuracy',
                       'total_num_studs': 'mae',  # Mean Absolute Error for regression tasks
                       'holddown_model': 'mae',
                       'tx': 'mae',
                       'ty': 'mae'})

# Train the model
model.fit(X_train, {'nail_spacing': Y_train_onehot['Nail spacing [cm]'], 
                    'num_sheathing_panels': Y_train['Number sheathing panels'],
                    'num_end_studs': Y_train['Number end studs'],
                    'total_num_studs': Y_train['Total number studs'],
                    'holddown_model': Y_train['HoldDown Model'],
                    'tx': Y_train['Tx(s)'],
                    'ty': Y_train['Ty(s)']},
          epochs=100, batch_size=32, validation_split=0.2)


# Evaluate the model on the test set
model.evaluate(X_test, {'nail_spacing': Y_test['nail_spacing'], 
                        'num_sheathing_panels': Y_test['num_sheathing_panels'],
                        'num_end_studs': Y_test['num_end_studs'],
                        'total_num_studs': Y_test['total_num_studs'],
                        'holddown_model': Y_test['holddown_model'],
                        'tx': Y_test['tx'],
                        'ty': Y_test['ty']})


IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [7]:
def label_encoder(Y_train, Y_test):
    label_encoder_nail_spacing = LabelEncoder()
    Y_train['Nail spacing [cm]'] = label_encoder_nail_spacing.fit_transform(Y_train['Nail spacing [cm]'])
    Y_test['Nail spacing [cm]'] = label_encoder_nail_spacing.transform(Y_test['Nail spacing [cm]'])
    Y_train_onehot_nail_spacing = to_categorical(Y_train['Nail spacing [cm]'], num_classes=3)
    Y_test_onehot_nail_spacing = to_categorical(Y_test['Nail spacing [cm]'], num_classes=3)

    label_encoder_num_sheathing_panels = LabelEncoder()
    Y_train['Number sheathing panels'] = label_encoder_num_sheathing_panels.fit_transform(Y_train['Number sheathing panels'])
    Y_test['Number sheathing panels'] = label_encoder_num_sheathing_panels.transform(Y_test['Number sheathing panels'])
    Y_train_onehot_num_sheathing_panels = to_categorical(Y_train['Number sheathing panels'], num_classes=2)
    Y_test_onehot_num_sheathing_panels = to_categorical(Y_test['Number sheathing panels'], num_classes=2)

    label_encoder_num_end_studs = LabelEncoder()
    Y_train['Number end studs'] = label_encoder_num_end_studs.fit_transform(Y_train['Number end studs'])
    Y_test['Number end studs'] = label_encoder_num_end_studs.transform(Y_test['Number end studs'])
    Y_train_onehot_num_end_studs = to_categorical(Y_train['Number end studs'], num_classes=6)
    Y_test_onehot_num_end_studs = to_categorical(Y_test['Number end studs'], num_classes=6)

    return Y_train_onehot_nail_spacing, Y_test_onehot_nail_spacing, Y_train_onehot_num_sheathing_panels, Y_test_onehot_num_sheathing_panels, Y_train_onehot_num_end_studs, Y_test_onehot_num_end_studs, Y_train, Y_test

In [40]:
input_layer = Input(shape=(input_shape,))

# One hot encoding Y
Y_train_onehot_nail_spacing, Y_test_onehot_nail_spacing, Y_train_onehot_num_sheathing_panels, Y_test_onehot_num_sheathing_panels, Y_train_onehot_num_end_studs, Y_test_onehot_num_end_studs, Y_train, Y_test = label_encoder(Y_train, Y_test)

# Define separate branches for each output
nail_spacing_output = Dense(3, activation='softmax', name='nail_spacing')(input_layer)
num_sheathing_panels_output = Dense(2, activation='softmax', name='num_sheathing_panels')(input_layer)
num_end_studs_output = Dense(6, activation='softmax', name='num_end_studs')(input_layer)
total_num_studs_output = Dense(1, activation='linear', name='total_num_studs')(input_layer)
holddown_model_output = Dense(1, activation='linear', name='holddown_model')(input_layer)
tx_output = Dense(1, activation='linear', name='tx')(input_layer)
ty_output = Dense(1, activation='linear', name='ty')(input_layer)

# Combine the outputs into a single model
model = Model(inputs=input_layer, outputs=[nail_spacing_output, num_sheathing_panels_output, 
                                           num_end_studs_output, total_num_studs_output, 
                                           holddown_model_output, tx_output, ty_output])

# Compile the model
model.compile(optimizer='adam', 
              loss={'nail_spacing': 'categorical_crossentropy',
                    'num_sheathing_panels': 'categorical_crossentropy',
                    'num_end_studs': 'categorical_crossentropy',
                    'total_num_studs': 'mean_squared_error',
                    'holddown_model': 'mean_squared_error',
                    'tx': 'mean_squared_error',
                    'ty': 'mean_squared_error'},
              metrics={'nail_spacing': 'accuracy',
                       'num_sheathing_panels': 'accuracy',
                       'num_end_studs': 'accuracy',
                       'total_num_studs': 'mae',
                       'holddown_model': 'mae',
                       'tx': 'mae',
                       'ty': 'mae'})

# Train the model
model.fit(X_train, {'nail_spacing': Y_train_onehot_nail_spacing, 
                    'num_sheathing_panels': Y_train_onehot_num_sheathing_panels,
                    'num_end_studs': Y_train_onehot_num_end_studs,
                    'total_num_studs': Y_train['Total number studs'],
                    'holddown_model': Y_train['HoldDown Model'],
                    'tx': Y_train['Tx(s)'],
                    'ty': Y_train['Ty(s)']},
          epochs=100, batch_size=32, validation_split=0.2)



Epoch 1/100
1246/1246 [==============================] - 10s 5ms/step - loss: 96.6684 - nail_spacing_loss: 0.0991 - num_sheathing_panels_loss: 0.3223 - num_end_studs_loss: 1.0217 - total_num_studs_loss: 64.6923 - holddown_model_loss: 30.4776 - tx_loss: 0.0293 - ty_loss: 0.0261 - nail_spacing_accuracy: 0.9899 - num_sheathing_panels_accuracy: 0.8697 - num_end_studs_accuracy: 0.6442 - total_num_studs_mae: 6.3216 - holddown_model_mae: 3.4358 - tx_mae: 0.1115 - ty_mae: 0.1016 - val_loss: 47.8061 - val_nail_spacing_loss: 0.0081 - val_num_sheathing_panels_loss: 0.2715 - val_num_end_studs_loss: 0.7845 - val_total_num_studs_loss: 24.9474 - val_holddown_model_loss: 21.7844 - val_tx_loss: 0.0057 - val_ty_loss: 0.0046 - val_nail_spacing_accuracy: 1.0000 - val_num_sheathing_panels_accuracy: 0.8893 - val_num_end_studs_accuracy: 0.7330 - val_total_num_studs_mae: 3.4801 - val_holddown_model_mae: 3.1597 - val_tx_mae: 0.0598 - val_ty_mae: 0.0532
Epoch 2/100
1246/1246 [==============================] - 6

In [42]:
# Evaluate the model on the test set
evaluation_results = model.evaluate(X_test, {'nail_spacing': Y_test_onehot_nail_spacing, 
                                             'num_sheathing_panels': Y_test_onehot_num_sheathing_panels,
                                             'num_end_studs': Y_test_onehot_num_end_studs,
                                             'total_num_studs': Y_test['Total number studs'],
                                             'holddown_model': Y_test['HoldDown Model'],
                                             'tx': Y_test['Tx(s)'],
                                             'ty': Y_test['Ty(s)']},
                                    batch_size=32)

# Extract and print the accuracy for each output
output_metrics = {name: value for name, value in zip(model.metrics_names, evaluation_results)}
print("Metrics on Test Set:")
for output_name, metric_value in output_metrics.items():
    print(f"{output_name}: {metric_value}")


390/390 [==============================] - 1s 3ms/step - loss: 15.5008 - nail_spacing_loss: 0.0000e+00 - num_sheathing_panels_loss: 0.2443 - num_end_studs_loss: 0.3613 - total_num_studs_loss: 2.0881 - holddown_model_loss: 12.7973 - tx_loss: 0.0055 - ty_loss: 0.0043 - nail_spacing_accuracy: 1.0000 - num_sheathing_panels_accuracy: 0.8929 - num_end_studs_accuracy: 0.8455 - total_num_studs_mae: 0.9306 - holddown_model_mae: 2.6309 - tx_mae: 0.0583 - ty_mae: 0.0514
Metrics on Test Set:
loss: 15.500787734985352
nail_spacing_loss: 0.0
num_sheathing_panels_loss: 0.2442822903394699
num_end_studs_loss: 0.36130228638648987
total_num_studs_loss: 2.088104009628296
holddown_model_loss: 12.797307968139648
tx_loss: 0.005488893017172813
ty_loss: 0.004300387110561132
nail_spacing_accuracy: 1.0
num_sheathing_panels_accuracy: 0.8928772211074829
num_end_studs_accuracy: 0.8454990983009338
total_num_studs_mae: 0.9306480884552002
holddown_model_mae: 2.630870819091797
tx_mae: 0.058348290622234344
ty_mae: 0.0514

### 2. 
- Try different regression models to improve regression loss

#### Predicting Values of C

In [31]:
path = './Files/After_Feature_Engineering/Split/'

X_train_C_1 = pd.read_csv(path + "X_train_C_part1_FE.csv", low_memory=False)
X_test_C_1 = pd.read_csv(path + "X_test_C_part1_FE.csv", low_memory=False)

Y_train_C_1 = pd.read_csv(path + "Y_train_C_part1_FE.csv", low_memory=False)
Y_test_C_1 = pd.read_csv(path + "Y_test_C_part1_FE.csv", low_memory=False)

In [32]:
# Define the input layer
input_layer_C_1 = Input(shape=(X_train_C_1.shape[1],))

# One hot encoding Y
Y_train_onehot_nail_spacing, Y_test_onehot_nail_spacing, Y_train_onehot_num_sheathing_panels, Y_test_onehot_num_sheathing_panels, Y_train_onehot_num_end_studs, Y_test_onehot_num_end_studs, Y_train_C_1, Y_test_C_1 = label_encoder(Y_train_C_1, Y_test_C_1)


nail_spacing_output = Dense(3, activation='softmax', name='nail_spacing')(input_layer_C_1)
num_sheathing_panels_output = Dense(2, activation='softmax', name='num_sheathing_panels')(input_layer_C_1)
num_end_studs_output = Dense(6, activation='softmax', name='num_end_studs')(input_layer_C_1)
total_num_studs_output = Dense(1, activation='linear', name='total_num_studs')(input_layer_C_1)
holddown_model_output = Dense(1, activation='linear', name='holddown/ats')(input_layer_C_1)

# Combine the outputs into a single model
model_C_1 = Model(inputs=input_layer_C_1, outputs=[nail_spacing_output, num_sheathing_panels_output, 
                                           num_end_studs_output, total_num_studs_output, 
                                           holddown_model_output])

# Compile the model
model_C_1.compile(optimizer='adam', 
              loss={'nail_spacing': 'categorical_crossentropy',
                    'num_sheathing_panels': 'categorical_crossentropy',
                    'num_end_studs': 'categorical_crossentropy',
                    'total_num_studs': 'mean_squared_error',
                    'holddown/ats': 'mean_squared_error'
                    },
              metrics={'nail_spacing': 'accuracy',
                       'num_sheathing_panels': 'accuracy',
                       'num_end_studs': 'accuracy',
                       'total_num_studs': 'mae',
                       'holddown/ats': 'mae'
                       })

# Train the model
model_C_1.fit(X_train_C_1, {'nail_spacing': Y_train_onehot_nail_spacing, 
                    'num_sheathing_panels': Y_train_onehot_num_sheathing_panels,
                    'num_end_studs': Y_train_onehot_num_end_studs,
                    'total_num_studs': Y_train_C_1['Total number studs'],
                    'holddown/ats': Y_train_C_1['HoldDown Model / ATS']
                    },
          epochs=100, batch_size=32, validation_split=0.2)

Epoch 1/100
1246/1246 [==============================] - 11s 7ms/step - loss: 100.5751 - nail_spacing_loss: 0.7274 - num_sheathing_panels_loss: 0.3666 - num_end_studs_loss: 0.9955 - total_num_studs_loss: 65.4998 - holddown/ats_loss: 32.9860 - nail_spacing_accuracy: 0.7029 - num_sheathing_panels_accuracy: 0.8398 - num_end_studs_accuracy: 0.6566 - total_num_studs_mae: 6.4154 - holddown/ats_mae: 3.5677 - val_loss: 48.6549 - val_nail_spacing_loss: 0.6451 - val_num_sheathing_panels_loss: 0.2823 - val_num_end_studs_loss: 0.7689 - val_total_num_studs_loss: 24.1488 - val_holddown/ats_loss: 22.8097 - val_nail_spacing_accuracy: 0.7405 - val_num_sheathing_panels_accuracy: 0.8894 - val_num_end_studs_accuracy: 0.7451 - val_total_num_studs_mae: 3.4345 - val_holddown/ats_mae: 3.2422
Epoch 2/100
1246/1246 [==============================] - 8s 6ms/step - loss: 40.7549 - nail_spacing_loss: 0.6195 - num_sheathing_panels_loss: 0.2671 - num_end_studs_loss: 0.7129 - total_num_studs_loss: 18.7989 - holddown/

In [12]:
# Evaluate the model on the test set
evaluation_results = model_C_1.evaluate(X_test_C_1, {'nail_spacing': Y_test_onehot_nail_spacing, 
                                             'num_sheathing_panels': Y_test_onehot_num_sheathing_panels,
                                             'num_end_studs': Y_test_onehot_num_end_studs,
                                             'total_num_studs': Y_test_C_1['Total number studs'],
                                             'holddown/ats': Y_test_C_1['HoldDown Model / ATS']
                                             },
                                    batch_size=32)

# Extract and print the accuracy for each output
output_metrics = {name: value for name, value in zip(model_C_1.metrics_names, evaluation_results)}
print("Metrics on Test Set:")
for output_name, metric_value in output_metrics.items():
    print(f"{output_name}: {metric_value}")

390/390 [==============================] - 1s 3ms/step - loss: 16.0792 - nail_spacing_loss: 0.5877 - num_sheathing_panels_loss: 0.2440 - num_end_studs_loss: 0.3573 - total_num_studs_loss: 2.0877 - holddown/ats_loss: 12.8025 - nail_spacing_accuracy: 0.7528 - num_sheathing_panels_accuracy: 0.8918 - num_end_studs_accuracy: 0.8500 - total_num_studs_mae: 0.9390 - holddown/ats_mae: 2.6588
Metrics on Test Set:
loss: 16.07921028137207
nail_spacing_loss: 0.5876692533493042
num_sheathing_panels_loss: 0.24397161602973938
num_end_studs_loss: 0.3573315739631653
total_num_studs_loss: 2.0877490043640137
holddown/ats_loss: 12.802483558654785
nail_spacing_accuracy: 0.7528306245803833
num_sheathing_panels_accuracy: 0.8918333053588867
num_end_studs_accuracy: 0.8499959707260132
total_num_studs_mae: 0.9390494227409363
holddown/ats_mae: 2.6588473320007324


#### Predicting values of Tx and Ty

In [15]:
X_train_C_2 = pd.read_csv(path + "X_train_C_part2_FE.csv", low_memory=False)
X_test_C_2 = pd.read_csv(path + "X_test_C_part2_FE.csv", low_memory=False)

Y_train_C_2 = pd.read_csv(path + "Y_train_C_part2_FE.csv", low_memory=False)
Y_test_C_2 = pd.read_csv(path + "Y_test_C_part2_FE.csv", low_memory=False)

In [21]:
# Define the input layer
input_layer_C_2 = Input(shape=(X_train_C_2.shape[1],))

# Define separate branches for each output
tx_output = Dense(1, activation='linear', name='tx')(input_layer_C_2)
ty_output = Dense(1, activation='linear', name='ty')(input_layer_C_2)

# Combine the outputs into a single model
model_C_2 = Model(inputs=input_layer_C_2, outputs=[tx_output, ty_output])

# Compile the model
model_C_2.compile(optimizer='adam', 
              loss={
                    'tx': 'mean_squared_error',
                    'ty': 'mean_squared_error'
                    },
              metrics={
                       'tx': 'mae',
                       'ty': 'mae'
                       })

# Train the model
model_C_2.fit(X_train_C_2, {
                        'tx': Y_train_C_2['Tx(s)'],
                        'ty': Y_train_C_2['Ty(s)']
                        },
          epochs=100, batch_size=32, validation_split=0.2)


Epoch 1/100
4/4 [==============================] - 1s 167ms/step - loss: 1.0084 - tx_loss: 0.5944 - ty_loss: 0.4140 - tx_mae: 0.6363 - ty_mae: 0.5328 - val_loss: 0.6280 - val_tx_loss: 0.3468 - val_ty_loss: 0.2812 - val_tx_mae: 0.5111 - val_ty_mae: 0.4171
Epoch 2/100
4/4 [==============================] - 0s 22ms/step - loss: 0.6388 - tx_loss: 0.4619 - ty_loss: 0.1770 - tx_mae: 0.5892 - ty_mae: 0.3233 - val_loss: 0.2026 - val_tx_loss: 0.1467 - val_ty_loss: 0.0559 - val_tx_mae: 0.3211 - val_ty_mae: 0.1853
Epoch 3/100
4/4 [==============================] - 0s 20ms/step - loss: 0.2006 - tx_loss: 0.0966 - ty_loss: 0.1040 - tx_mae: 0.2519 - ty_mae: 0.2776 - val_loss: 0.2603 - val_tx_loss: 0.1823 - val_ty_loss: 0.0780 - val_tx_mae: 0.3691 - val_ty_mae: 0.2270
Epoch 4/100
4/4 [==============================] - 0s 18ms/step - loss: 0.2470 - tx_loss: 0.1897 - ty_loss: 0.0573 - tx_mae: 0.3822 - ty_mae: 0.1869 - val_loss: 0.2017 - val_tx_loss: 0.1131 - val_ty_loss: 0.0886 - val_tx_mae: 0.2896 - va

In [23]:
# Evaluate the model on the test set
evaluation_results = model_C_2.evaluate(X_test_C_2, {
                                                    'tx': Y_test_C_2['Tx(s)'],
                                                    'ty': Y_test_C_2['Ty(s)']
                                             },
                                    batch_size=32)

# Extract and print the accuracy for each output
output_metrics = {name: value for name, value in zip(model_C_2.metrics_names, evaluation_results)}
print("Metrics on Test Set:")
for output_name, metric_value in output_metrics.items():
    print(f"{output_name}: {metric_value}")

2/2 [==============================] - 0s 3ms/step - loss: 0.0102 - tx_loss: 0.0064 - ty_loss: 0.0038 - tx_mae: 0.0619 - ty_mae: 0.0502
Metrics on Test Set:
loss: 0.010217642411589622
tx_loss: 0.006403851322829723
ty_loss: 0.003813790623098612
tx_mae: 0.06187506765127182
ty_mae: 0.050161369144916534


#### Predicting values of D 

In [27]:
X_train_D = df = pd.read_csv(path + "X_train_D_FE.csv", low_memory=False)
X_test_D = pd.read_csv(path + "X_test_D_FE.csv", low_memory=False)

Y_train_D = df = pd.read_csv(path + "Y_train_d_FE.csv", low_memory=False)
Y_test_D = pd.read_csv(path + "Y_test_D_FE.csv", low_memory=False)

# Chnage the column name
column_names = ['omega_x', 'omega_y', 'mu_x', 'mu_y', 'cmr', 'ssf','acmr', 'io-ln', 'io-b', 'ls-ln', 'ls-b', 'cp-ln', 'cp-b']
Y_train_D.columns = column_names
Y_test_D.columns = column_names

In [44]:
# Define the input layer
input_layer_D = Input(shape=(X_train_D.shape[1],))

#Add a layer
#input_layer_D = Dense(8966, activation='relu')(input_layer_D)

# Define separate branches for each output
omega_x_output = Dense(1, activation='linear', name='omega_x')(input_layer_D)
omega_y_output = Dense(1, activation='linear', name='omega_y')(input_layer_D)
mu_x_output = Dense(1, activation='linear', name='mu_x')(input_layer_D)
mu_y_output = Dense(1, activation='linear', name='mu_y')(input_layer_D)
cmr_output = Dense(1, activation='linear', name='cmr')(input_layer_D)
ssf_output = Dense(1, activation='linear', name='ssf')(input_layer_D)
acmr_output = Dense(1, activation='linear', name='acmr')(input_layer_D)
io_ln_output = Dense(1, activation='linear', name='io-ln')(input_layer_D)
io_b_output = Dense(1, activation='linear', name='io-b')(input_layer_D)
ls_ln_output = Dense(1, activation='linear', name='ls-ln')(input_layer_D)
ls_b_output = Dense(1, activation='linear', name='ls-b')(input_layer_D)
cp_ln_output = Dense(1, activation='linear', name='cp-ln')(input_layer_D)
cp_b_output = Dense(1, activation='linear', name='cp-b')(input_layer_D)

# Combine the outputs into a single model
model_D = Model(inputs=input_layer_D, outputs=[omega_x_output, omega_y_output, 
                                               mu_x_output, mu_y_output,
                                               cmr_output, ssf_output, acmr_output,
                                               io_ln_output, io_b_output,
                                               ls_ln_output, ls_b_output,
                                               cp_ln_output, cp_b_output])

# Compile the model
model_D.compile(optimizer='adam', 
              loss={
                        'omega_x': 'mean_squared_error',
                        'omega_y': 'mean_squared_error',
                        'mu_x': 'mean_squared_error',
                        'mu_y': 'mean_squared_error',
                        'cmr': 'mean_squared_error',
                        'ssf': 'mean_squared_error',
                        'acmr': 'mean_squared_error',
                        'io-ln': 'mean_squared_error',
                        'io-b': 'mean_squared_error',
                        'ls-ln': 'mean_squared_error',
                        'ls-b': 'mean_squared_error',
                        'cp-ln': 'mean_squared_error',
                        'cp-b': 'mean_squared_error'
                    },
              metrics={
                       'omega_x': 'mae',
                        'omega_y': 'mae',
                        'mu_x': 'mae',
                        'mu_y': 'mae',
                        'cmr': 'mae',
                        'ssf': 'mae',
                        'acmr': 'mae',
                        'io-ln': 'mae',
                        'io-b': 'mae',
                        'ls-ln': 'mae',
                        'ls-b': 'mae',
                        'cp-ln': 'mae',
                        'cp-b': 'mae'
                       })

# Train the model
model_D.fit(X_train_D, {
                        'omega_x': Y_train_D['omega_x'],
                        'omega_y': Y_train_D['omega_y'],
                        'mu_x': Y_train_D['mu_x'],
                        'mu_y': Y_train_D['mu_y'],
                        'cmr': Y_train_D['cmr'],
                        'ssf': Y_train_D['ssf'],
                        'acmr': Y_train_D['acmr'],
                        'io-ln': Y_train_D['io-ln'],
                        'io-b': Y_train_D['io-b'],
                        'ls-ln': Y_train_D['ls-ln'],
                        'ls-b': Y_train_D['ls-b'],
                        'cp-ln': Y_train_D['cp-ln'],
                        'cp-b': Y_train_D['cp-b']
                        
                        },
          epochs=100, batch_size=32, validation_data=(X_test_D, {
                                                            'omega_x': Y_test_D['omega_x'],
                                                            'omega_y': Y_test_D['omega_y'],
                                                            'mu_x': Y_test_D['mu_x'],
                                                            'mu_y': Y_test_D['mu_y'],
                                                            'cmr': Y_test_D['cmr'],
                                                            'ssf': Y_test_D['ssf'],
                                                            'acmr': Y_test_D['acmr'],
                                                            'io-ln': Y_test_D['io-ln'],
                                                            'io-b': Y_test_D['io-b'],
                                                            'ls-ln': Y_test_D['ls-ln'],
                                                            'ls-b': Y_test_D['ls-b'],
                                                            'cp-ln': Y_test_D['cp-ln'],
                                                            'cp-b': Y_test_D['cp-b']
                                                            }
                                                  ))

Epoch 1/100
5/5 [==============================] - 6s 522ms/step - loss: 79.7344 - omega_x_loss: 23.0440 - omega_y_loss: 19.9217 - mu_x_loss: 8.5898 - mu_y_loss: 7.2040 - cmr_loss: 5.1818 - ssf_loss: 1.0122 - acmr_loss: 5.3730 - io-ln_loss: 1.2177 - io-b_loss: 1.6888 - ls-ln_loss: 1.9530 - ls-b_loss: 1.6345 - cp-ln_loss: 1.1029 - cp-b_loss: 1.8109 - omega_x_mae: 3.7212 - omega_y_mae: 3.4430 - mu_x_mae: 2.3923 - mu_y_mae: 2.2595 - cmr_mae: 1.9273 - ssf_mae: 0.9009 - acmr_mae: 1.9675 - io-ln_mae: 0.9406 - io-b_mae: 1.1083 - ls-ln_mae: 1.1730 - ls-b_mae: 1.0680 - cp-ln_mae: 0.9172 - cp-b_mae: 1.1121 - val_loss: 65.1635 - val_omega_x_loss: 17.9581 - val_omega_y_loss: 16.6288 - val_mu_x_loss: 8.0974 - val_mu_y_loss: 7.5204 - val_cmr_loss: 3.0593 - val_ssf_loss: 0.8058 - val_acmr_loss: 2.9705 - val_io-ln_loss: 1.0902 - val_io-b_loss: 1.4163 - val_ls-ln_loss: 1.7626 - val_ls-b_loss: 1.3171 - val_cp-ln_loss: 0.9781 - val_cp-b_loss: 1.5589 - val_omega_x_mae: 3.6007 - val_omega_y_mae: 3.6047 - v

In [46]:
# Evaluate the model on the test set
evaluation_results = model_D.evaluate(X_test_D, {
                                                    'omega_x': Y_test_D['omega_x'],
                                                    'omega_y': Y_test_D['omega_y'],
                                                    'mu_x': Y_test_D['mu_x'],
                                                    'mu_y': Y_test_D['mu_y'],
                                                    'cmr': Y_test_D['cmr'],
                                                    'ssf': Y_test_D['ssf'],
                                                    'acmr': Y_test_D['acmr'],
                                                    'io-ln': Y_test_D['io-ln'],
                                                    'io-b': Y_test_D['io-b'],
                                                    'ls-ln': Y_test_D['ls-ln'],
                                                    'ls-b': Y_test_D['ls-b'],
                                                    'cp-ln': Y_test_D['cp-ln'],
                                                    'cp-b': Y_test_D['cp-b']
                                             },
                                    batch_size=32)

# Extract and print the accuracy for each output
output_metrics = {name: value for name, value in zip(model_D.metrics_names, evaluation_results)}
print("Metrics on Test Set:")
for output_name, metric_value in output_metrics.items():
    print(f"{output_name}: {metric_value}")

2/2 [==============================] - 0s 14ms/step - loss: 6.9044 - omega_x_loss: 2.7114 - omega_y_loss: 2.9292 - mu_x_loss: 0.1931 - mu_y_loss: 0.1878 - cmr_loss: 0.2907 - ssf_loss: 0.0132 - acmr_loss: 0.5049 - io-ln_loss: 0.0172 - io-b_loss: 0.0088 - ls-ln_loss: 0.0149 - ls-b_loss: 0.0056 - cp-ln_loss: 0.0217 - cp-b_loss: 0.0058 - omega_x_mae: 1.1205 - omega_y_mae: 1.3178 - mu_x_mae: 0.3177 - mu_y_mae: 0.2682 - cmr_mae: 0.4373 - ssf_mae: 0.0884 - acmr_mae: 0.5768 - io-ln_mae: 0.0900 - io-b_mae: 0.0612 - ls-ln_mae: 0.0822 - ls-b_mae: 0.0559 - cp-ln_mae: 0.1056 - cp-b_mae: 0.0567
Metrics on Test Set:
loss: 6.904378414154053
omega_x_loss: 2.7114062309265137
omega_y_loss: 2.9292235374450684
mu_x_loss: 0.19309023022651672
mu_y_loss: 0.18782052397727966
cmr_loss: 0.29071465134620667
ssf_loss: 0.01321420818567276
acmr_loss: 0.5049476623535156
io-ln_loss: 0.01715112291276455
io-b_loss: 0.008800121955573559
ls-ln_loss: 0.014853449538350105
ls-b_loss: 0.0056437342427670956
cp-ln_loss: 0.02173